## Notebook 4: 04_final_analysis_and_reporting.ipynb

#### Author: Satveer Kaur
#### Date: 2025-10-28

#### Notebook Purpose:
This notebook represents the **Final Analysis, Validation, and Reporting** phase of the data science pipeline. It integrates the final risk segments developed on the sample data (Notebook 03) and applies them to the full dataset.

**Primary Objective:** To finalize the data preparation for external consumption (Tableau/BI) and to generate the core executive summary metrics.

**Key Deliverables:**
1. **Executive Risk Tables:** Generate three summary tables (FICO, DTI, Income) showing Observed Default Rate (ODR) and Portfolio Volume % for executive review.
2. **Data Finalization:** Re-apply validated risk binning logic to the entire cleaned loan portfolio.
3. **Export:** Export the complete, segmented data source for dashboard creation in Tableau or other Business Intelligence tools.

**Input:** `binned_sample_for_reporting.csv` (Sample with segments for table generation) and `clean_data_for_sampling.csv` (Full cleaned data for Tableau source creation).

**Output:** `final_tableau_source.csv` (Full segmented data for BI).

#### 1. Setup and Data Loading

In [9]:
# Importing Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100

df_report = pd.read_csv('../data/processed/binned_sample_for_reporting.csv', low_memory=False, parse_dates=['issue_date'])

print(f'Loaded {len(df_report)} records for final reporting.')
df_report.head()

Loaded 226066 records for final reporting.


,id,amount_requested,funded_amount,funded_amount_invested,term,interest_rate,installment,grade,sub_grade,emp_title,...,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,is_default,fico_score,fico_group,dti_quintile,income_group
0,133947999,25000.0,25000.0,25000.0,60 months,11.05,544.19,B,B4,recruiting,...,0.0,160895.0,100663.0,60100.0,96095.0,0,712.0,FICO_Good (670-739),DTI_Q2,$50k - $100k
1,140850180,15000.0,15000.0,15000.0,36 months,16.14,528.40,C,C4,NaN,...,0.0,77336.0,63916.0,3700.0,50436.0,0,662.0,FICO_Subprime/Poor (<670),DTI_Q5 (Highest Risk),$50k - $100k
2,76293403,36000.0,36000.0,36000.0,36 months,5.32,1084.14,A,A1,Lt Colonel,...,0.0,334249.0,61076.0,97000.0,24000.0,0,747.0,FICO_Very Good (740-799),DTI_Q2,> $150k
3,95103209,5925.0,5925.0,5925.0,36 months,12.74,198.90,C,C1,president,...,0.0,71000.0,4956.0,5200.0,0.0,0,662.0,FICO_Subprime/Poor (<670),DTI_Q1 (Lowest Risk),$50k - $100k
4,2934677,19650.0,19650.0,19650.0,36 months,13.11,663.13,B,B4,Akal Security INC,...,0.0,172059.0,158860.0,12700.0,148759.0,0,707.0,FICO_Good (670-739),DTI_Q5 (Highest Risk),$100k - $150k


#### 2. Analytical Utility: `create_risk_table()` Function
**Purpose:** To define a standardized, reusable function for generating the final, formatted executive risk tables. This utility takes a risk segment column (e.g., `fico_group`) and calculates the **Volume (n), Volume Percentage, and the Observed Default Rate (ODR)**, ensuring consistent and professional output for the final report.

In [10]:
def create_risk_table(df, group_col, target_col='is_default'):
    """
    Generates the final risk profile table: Volume, Volume %, and Observed Default Rate (ODR).
    
    Args:
        df (pd.DataFrame): The input DataFrame.
        group_col (str): The column containing the risk segments (e.g., 'fico_group').
        target_col (str): The binary target column ('is_default').
        
    Returns:
        pd.DataFrame: A formatted table ready for reporting.
    """
    # Calculate ODR and Volume 
    risk_data = df.groupby(group_col, observed=True)[target_col].agg(
        volume = ('count'),
        ODR = ('mean')
    ).reset_index()

    # Calculate Volume Percentage
    total_volume = risk_data['volume'].sum()
    risk_data['volume_pct'] = risk_data['volume'] / total_volume

    # Rename and select final columns
    risk_data.columns = [
        'Risk Segment',
        'Total Volume',
        'ODR (Observed Default Rate)',
        'Volume % of Portfolio'
    ]

    # set the Segment table as index
    risk_data = risk_data.set_index('Risk Segment')

    # Format the table 
    risk_data = risk_data.style.format({
        'Total Volume' : '{:,.0f}'.format,
        'ODR (Observed Default Rate)' : '{:.2%}'.format,
        'Volume % of Portfolio' : '{:.2%}'.format
    })

    return risk_data

#### 3. Establish Categorical Order
**Purpose:** Re-applying the ordinal sequence to ensure correct sorting of tables and plots (Lowest Risk -> Highest Risk).

In [11]:
# FICO: High score = Low Risk
fico_categories = [
    'FICO_Excellent (800+)', 
    'FICO_Very Good (740-799)', 
    'FICO_Good (670-739)', 
    'FICO_Subprime/Poor (<670)'
]

# DTI: Low DTI = Low Risk
dti_categories = [
    'DTI_Q1 (Lowest Risk)', 
    'DTI_Q2', 
    'DTI_Q3', 
    'DTI_Q4', 
    'DTI_Q5 (Highest Risk)'
]

# INCOME: High Income = Low Risk
income_categories = [
    '> $150k', 
    '$100k - $150k', 
    '$50k - $100k', 
    '< $50k'
]

category_mappings = [
    ('fico_group', fico_categories),
    ('dti_quintile', dti_categories),
    ('income_group', income_categories)
]

# Applying the defined order to df
for col_name, order_list in category_mappings:
    df_report[col_name] = pd.Categorical(
        df_report[col_name],
        categories=order_list,
        ordered=True
    )

#### 4. Generate Final Executive Risk Tables
**Purpose:** This section executes the standardized `create_risk_table` utility to produce the three primary deliverables for the executive report. By using the **ordered categorical features** (established in Section 2.5), the analysis ensures these tables are sorted correctly by **monotonic risk trend** (Lowest Risk to Highest Risk).
Each table quantifies the **Observed Default Rate (ODR)** against the **Portfolio Volume %** to provide a complete picture of risk exposure and severity.

In [12]:
# FICO Group Risk Table
fico_risk_table = create_risk_table(df_report, 'fico_group')
fico_risk_table

,Total Volume,ODR (Observed Default Rate),Volume % of Portfolio
Risk Segment,,,
FICO_Excellent (800+),"3,147",3.21%,1.39%
FICO_Very Good (740-799),"24,968",5.16%,11.04%
FICO_Good (670-739),"161,544",11.91%,71.46%
FICO_Subprime/Poor (<670),"36,407",17.32%,16.10%


In [13]:
# DTI Quintile Risk Table
dti_risk_table = create_risk_table(df_report, 'dti_quintile')
dti_risk_table

,Total Volume,ODR (Observed Default Rate),Volume % of Portfolio
Risk Segment,,,
DTI_Q1 (Lowest Risk),"45,250",9.11%,20.03%
DTI_Q2,"45,122",10.37%,19.97%
DTI_Q3,"45,244",11.80%,20.03%
DTI_Q4,"45,160",13.15%,19.99%
DTI_Q5 (Highest Risk),"45,121",15.20%,19.97%


In [14]:
# Annual Income Group Risk Table
income_risk_table = create_risk_table(df_report, 'income_group')
income_risk_table

,Total Volume,ODR (Observed Default Rate),Volume % of Portfolio
Risk Segment,,,
> $150k,"13,093",8.15%,5.86%
$100k - $150k,"33,476",9.59%,14.98%
$50k - $100k,"112,855",12.05%,50.52%
< $50k,"63,975",13.89%,28.64%


In [15]:
print('All three core risk tables are generated.')

All three core risk tables are generated.


#### 5. Final Data Prep and Export for Tableau Dashboard
**Purpose:** Load the ENTIRE cleaned dataset, apply the validated binning logic and export the full file, along with the summary tables, to a single Excel file.

In [16]:
# save the above generate tables to excel file
report_dataframes = {
    'FICO Risk Profile': fico_risk_table,
    'DTI Risk Profile': dti_risk_table,
    'Income Risk Profile': income_risk_table
}

output_file = '../data/processed/risk_segmentation_report.xlsx'

with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    for sheet_name, df_to_write in report_dataframes.items():
        df_to_write.to_excel(writer, sheet_name=sheet_name, index=False)

print('Executive Summary Tables saved to excel.')

# Load full clean data 
df_tableau = pd.read_csv('../data/processed/clean_data_for_sampling.csv', low_memory=False) # this was created in notebook 1

# Re-create the target variable (critical for Tableau calculations)
df_tableau['is_default'] =  df_tableau['loan_status'].apply(
    lambda x : 1 if x in ['Charged Off', 'Default', 'Does not meet the credit policy. Status:Charged Off'] else 0
)

# Re-apply Feature Engineering from Notebook 03 ---
print("Applying validated binning logic to the full dataset...")

# NOTE: These placeholder bins MUST match the logic used in Notebook 03.
# The max() + 1 ensures the upper bound is inclusive.

# Create a single FICO score by taking the mean of the high and low range
df_tableau['fico_score']= df_tableau[['fico_high', 'fico_low']].mean(axis=1)

# FICO BINNING 
fico_bins = [0, 670, 740, 800, df_tableau['fico_score'].max() + 1]
fico_labels = fico_categories # Reusing the categories defined in 3
df_tableau['fico_group'] = pd.cut(df_tableau['fico_score'], bins=fico_bins, labels=fico_labels, right=False)
    
# DTI QUINTILE BINNING 
dti_labels = dti_categories # Reusing the categories defined in 3
df_tableau['dti_quintile'] = pd.qcut(df_tableau['debt_to_income_ratio'], q=5, labels=dti_labels)

# INCOME BINNING
income_bins = [0, 50000, 100000, 150000, df_tableau['annual_income'].max() + 1]
income_labels = income_categories # Reusing the categories defined in 35
df_tableau['income_group'] = pd.cut(df_tableau['annual_income'], bins=income_bins, labels=income_labels, right=False)

# List of columns to keep in the final output
columns_to_keep = [
    # IDENTIFIERS & KEYS (For filtering/linking)
    'id', 
    'state',      # Needed for Geographic Risk analysis 

    # NUMERIC COLUMNS FOR VOLUME & KPI CALCULATIONS 
    'amount_requested',       # Principal loan amount (Needed for Volume KPI)
    'funded_amount',     # Amount actually funded (Alternative Volume KPI)
    'interest_rate',        # Interest rate (Needed for rate analysis)

    # TARGET & DATE VARIABLES (For KPI and Trend analysis)
    'loan_status',     # The target variable (Fully Paid / Default)
    'issue_date',      # Loan issue date (Needed for the ODR Trend line)
    'term',            # Needed for policy filtering
    'grade',           # Needed for policy filtering
    'purpose',
    # FINAL ENGINEERED FEATURES (The core of your analysis/model input)
    'fico_group',      # segmented FICO group
    'dti_quintile',    # segmented DTI group
    'income_group',    # segmented income group
    'purpose',    
    'is_default',      # Binary target variable (1/0)

    'recoveries',
    'total_rec_prncp'

]


# Export full data for tableau
df_tableau[columns_to_keep].to_csv('../data/processed/final_tableau_source.csv', index=False)
print('Full Data Source Exported to CSV.')

Executive Summary Tables saved to excel.
Applying validated binning logic to the full dataset...
Full Data Source Exported to CSV.


#### 6. Reporting Narrative and Executive Conclusions

##### Executive Summary of Key Risk Findings
Based on the analysis of the stratified loan portfolio sample, the following high-impact insights were confirmed:

1. **Monotonic Risk:** All three engineered risk features (FICO, DTI, and Income) maintain a strong, monotonic relationship with the Observed Default Rate (ODR), confirming their stability and predictive reliability.
2. **FICO vs. Exposure:** The riskiest segment, **FICO_Subprime/Poor (<670)**, exhibits the highest ODR, but the largest volume of loans still resides in the **FICO_Good (670-739)** segment. Policy focus should be placed on managing the boundary of the 'Good' segment.
3. **DTI Leverage:** Risk significantly accelerates in the **DTI_Q4** and **DTI_Q5 (Highest Risk)** segments. These two highest-leverage quintiles must be tightly controlled in underwriting policy.
4. **Income Impact:** The lowest income segment (**< $50k**) shows a substantially higher ODR, confirming income stability is a primary defense against default, independent of FICO score.

##### Actionable Policy Recommendations

1. **Tighten DTI Thresholds:** Review and potentially lower the maximum Debt-to-Income ratio allowed for applicants falling into the FICO 'Good' or 'Fair' segments, as high leverage combined with moderate credit quality is the primary risk accelerator.
2. **Portfolio Monitoring:** Use the exported data source to create a dashboard that continuously tracks loan volume growth against ODR performance in the **DTI_Q4** and **DTI_Q5** segments, setting strict growth limits for these categories.
3. **Cross-Feature Review:** Future modeling should prioritize features that interact strongly with the lowest income group to mitigate the high baseline risk observed in the '< $50k' category.
